In [1]:
import httplib2
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from parsel import Selector
from statistics import mean
import time
import requests
from datetime import datetime
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"}
http = httplib2.Http()
import mysql.connector

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from parsel import Selector
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

# configure webdriver
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen
# configure chrome browser to not load images and javascript
chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option(
    "prefs", {"profile.managed_default_content_settings.images": 2}
)

In [3]:
##create database
justwatch_db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="4156"
)

mycursor = justwatch_db.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS justwatch_movies_us_db")

In [ ]:
##connect to db
import pandas as pd
from sqlalchemy import create_engine

# Credentials to database connection
hostname="localhost"
dbname="justwatch_movies_us_db"
uname="root"
pwd="4156"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

In [4]:
base_url = "https://www.justwatch.com"

In [5]:
# Python3 Program to Create list 
# with integers within given range 
  
def createYears(r1, r2):
    return [item for item in range(r1, r2+1)]
      
genres = ['act','cmy',  'doc', 'fnt', 'hrr', 'msc', 'rms', 'spt', 'wsn', 'ani', 'crm', 'drm', 'hst', 'fml', 'trl', 'scf', 'war', 'rly']
years = createYears(1900, 2023)


search_links = []
for i in genres:
    for j in years:
        link = f"https://www.justwatch.com/us/movies?genres={i}&release_year_from={j}&release_year_until={j}"
        search_links.append(link)
        
len(search_links)

2232

In [6]:
titleLinks = []
for link in search_links:
    chrome_options = Options()  
    chromedriver = webdriver.Chrome(ChromeDriverManager().install())
    with chromedriver as browser:
        browser.get(link)
        time.sleep(5)

        last_height = browser.execute_script("return document.body.scrollHeight")
        SCROLL_PAUSE_TIME = 5
        while True:
                # Scroll down to bottom
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # Wait to load page
                time.sleep(SCROLL_PAUSE_TIME)

                # Calculate new scroll height and compare with last scroll height
                new_height = browser.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height
                
        page_source =browser.page_source
        soup=BeautifulSoup(page_source,'html.parser')
        productlist = soup.find("div",{"listlayout":"Grid"})
        titles = productlist.find_all("div", {"class":"title-list-grid__item"})
        for title in titles:
                    title_link = title.find("a").get('href')
                    title_link = base_url+title_link
                    titleLinks.append(title_link)

C:\Users\user\AppData\Local\Temp\ipykernel_13264\4293954353.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chromedriver = webdriver.Chrome(ChromeDriverManager().install())


WebDriverException: Message: chrome not reachable
  (Session info: chrome=104.0.5112.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x007478B3+2193587]
	Ordinal0 [0x006E0681+1771137]
	Ordinal0 [0x005F4070+802928]
	Ordinal0 [0x005E9AB2+760498]
	Ordinal0 [0x005DCEAF+708271]
	Ordinal0 [0x005E1142+725314]
	Ordinal0 [0x005E55C9+742857]
	Ordinal0 [0x005F5500+808192]
	Ordinal0 [0x0064D2C2+1168066]
	Ordinal0 [0x0063D5C6+1103302]
	Ordinal0 [0x006177E0+948192]
	Ordinal0 [0x006186E6+952038]
	GetHandleVerifier [0x009F0CB2+2738370]
	GetHandleVerifier [0x009E21B8+2678216]
	GetHandleVerifier [0x007D17AA+512954]
	GetHandleVerifier [0x007D0856+509030]
	Ordinal0 [0x006E743B+1799227]
	Ordinal0 [0x006EBB68+1817448]
	Ordinal0 [0x006EBC55+1817685]
	Ordinal0 [0x006F5230+1856048]
	BaseThreadInitThunk [0x75D8FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77477A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77477A6E+238]


In [ ]:
df_movie_links = pd.DataFrame(titleLinks, columns = ['Title Links'])
# Convert dataframe to sql table                                   
df_movie_links.to_sql('title_links', engine, if_exists='replace', index=False)

3614

In [ ]:
def get_detail(soup_object):
    labels  = []
    values = []
    table_info = soup_object.find("div", {"class":"title-info"})
    rows = table_info.find_all("div", {"class":"detail-infos"})
    for row in rows:
        label = row.find("div", {"class":"detail-infos__subheading"}).text.replace('\n',"")
        value = row.find("div", {"class":"detail-infos__value"}).text.replace('\n',"")
        labels.append(label)
        values.append(value)
        
    detail = {labels[i]: values[i] for i in range(len(labels))}
        
    return detail

def get_cast(soup_object):
    cast = []
    actors = soup_object.find_all("div", {"class":"title-credits__actor"})
    for actor in actors:
        actor_name = actor.text.replace('\n',"")
        cast.append(actor_name)
    return cast

class Pricing:
    def __init__(self, soup_object):
        self.soup_object = soup_object

    def get_stream_options(self):
        stream_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)


        return stream_list


    def get_rent_options(self):
        rent_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list

    def get_buy_options(self):
        buy_list = []
        price_comparison = self.soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list


In [ ]:
def get_stream_options(soup_object):
        stream_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_stream = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--stream price-comparison__grid__row--block"})
        for comp in price_comparison_stream:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for comp in comp:
                service = comp.find("img")
                service = service['title']
                pricing = comp.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                stream_options = {service:pricing}
                stream_list.append(stream_options)
        return stream_list


def get_rent_options(soup_object):
        rent_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_rent = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--rent price-comparison__grid__row--block"})
        for comp in price_comparison_rent:
            comp =  comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                rent_options = {service:pricing}
                rent_list.append(rent_options)
        
        return rent_list
    
def get_buy_options(soup_object):
        buy_list = []
        price_comparison = soup_object.find("div", {"class":"price-comparison--block"})
        price_comparison_buy = price_comparison.find_all("div", {"class":"price-comparison__grid__row price-comparison__grid__row--buy price-comparison__grid__row--block"})
        for comp in price_comparison_buy:
            comp = comp.find_all("div", {"class":"price-comparison__grid__row__element"})
            for element in comp:
                service = element.find("img")
                service = service['title']
                pricing = element.find("div", {"class":"price-comparison__grid__row__price"}).text.replace('\n',"")
                buy_options = {service:pricing}
                buy_list.append(buy_options)
                
        return buy_list

In [ ]:
def get_best(driver):
            driver.implicitly_wait(0.5)
            '''
            best = WebDriverWait(driver=driver, timeout=5).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[3]/div/div[2]/div/div[1]/div[1]/div/div[1]'))
            )
            best.click()
            
            time.sleep(5)
            '''
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_best = get_stream_options(hun)
            buy_best = get_buy_options(hun)
            rent_best = get_rent_options(hun)
            
            options = {"stream_best": stream_best,
                    "rent_best":rent_best,
                    "buy_best":buy_best}
            
            return options
        
def get_free(driver, bts):
            time.sleep(5)
            bts[1].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_free = get_stream_options(hun)
            buy_free = get_buy_options(hun)
            rent_free = get_rent_options(hun)
            
            
            options = {"stream_free": stream_free,
                    "rent_free":rent_free,
                    "buy_free":buy_free}
        
            return options
        
def get_sd(driver, bts):
            driver.implicitly_wait(0.5)
            bts[2].click()
            
            time.sleep(5)
            
            page_source =driver.page_source
            hun=BeautifulSoup(page_source,'html.parser')
            stream_sd = get_stream_options(hun)
            buy_sd = get_buy_options(hun)
            rent_sd = get_rent_options(hun)
            
            options = {"stream_sd": stream_sd,
                    "rent_sd":rent_sd,
                    "buy_sd":buy_sd}
        
            return options
        
def get_hd(driver, bts):
        driver.implicitly_wait(0.5)
            
        bts[3].click()        
        time.sleep(5)
            
        page_source =driver.page_source
        
        hun=BeautifulSoup(page_source,'html.parser')
        stream_hd = get_stream_options(hun)
        buy_hd = get_buy_options(hun)
        rent_hd = get_rent_options(hun)
            
        options = {"stream_hd": stream_hd,
                    "rent_hd":rent_hd,
                    "buy_hd":buy_hd}
        
        return options
        
      
def get_4k(driver, bts):   
        driver.implicitly_wait(0.5)
        bts[4].click()
            
        time.sleep(5)
            
        page_source =  driver.page_source
        hun=BeautifulSoup(page_source,'html.parser')
        stream_4k = get_stream_options(hun)
        buy_4k = get_buy_options(hun)
        rent_4k = get_rent_options(hun)       
            
        options = {"stream_4k": stream_4k,
                    "rent_4k":rent_4k,
                    "buy_4k":buy_4k}
        
        return options

In [ ]:
def get_data(titleLinks):
    data = []
    for link in titleLinks:
            driver = webdriver.Chrome(ChromeDriverManager().install())
            with driver as driver:
                driver.get(link)
                banner = WebDriverWait(driver=driver, timeout=5).until(
                    EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div[5]/div[2]/div[2]/button[1]'))
                    )
                
                banner.click()
                
                time.sleep(1)
                bts = driver.find_elements(By.CLASS_NAME,'jw-chip-button')
                
                time.sleep(1)
                try:
                    imdb_link = WebDriverWait(driver=driver, timeout=5).until(
                        EC.presence_of_element_located((By.XPATH, '//*[@id="base"]/div[2]/div/div[1]/div/aside/div[1]/div[3]/div[1]/div[2]/div/div[2]/a'))
                    )
               
                    imdb_link = imdb_link.get_attribute('href')
                except:
                    imdb_link = None
                
                page_source =  driver.page_source
                
                hun=BeautifulSoup(page_source,'html.parser')
                
                
                try:
                    detail = get_detail(hun)
                except:
                    detail = None
                    
                try:
                    title_name = hun.find("div", {"class":"title-block"})
                    title_name = title_name.find("h1").text.replace('\n',"")
                except:
                    title_name = None
                    
                try:
                    year = hun.find("span", {"class":"text-muted"}).text.replace('\n',"")
                except:
                    year = None
                
                try:
                    prices_best = get_best(driver)
                except:
                    prices_best = None
                
                try:
                    prices_free = get_free(driver,bts)
                except:
                    prices_free = None
                    
                try:
                    prices_sd = get_sd(driver, bts)
                except:
                    prices_sd = None
                    
                try:
                    prices_hd = get_hd(driver, bts)
                except:
                    prices_hd = None

                try:   
                    prices_4k = get_4k(driver, bts)
                except:
                    prices_4k = None
        
        
                title_url = link
                
                tv = {"title":title_name, "year":year, "detail":detail, "4k prices":prices_4k, "hd prices":prices_hd, "sd prices":prices_sd, "free":prices_free, "best prices":prices_best,"imdb_link":imdb_link,"tite url": title_url}
        
                data.append(tv)
                print(title_name)
        
    return  data
        
        
        

In [ ]:
Data = get_data(titleLinks)

C:\Users\user\AppData\Local\Temp\ipykernel_13556\1066804298.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


 Laal Zameen 
 The Mannequin 


def get_synopsis(soup_object):
    driver.get(soup_object)       
    synopsis = driver.find_element(By.XPATH, '//*[@id="base"]/div[2]/div/div[2]/div[2]/div[2]/div[2]/div[2]/p').text
    return synopsis

get_rent_options(soup)

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(Data)
df

,title,year,detail,4k prices,hd prices,sd prices,free,best prices,imdb_link,tite url
0,Laal Zameen,(2022),"{'Rating': ' 4% ', 'Genres': 'Action & Adventu...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [], 'rent_hd': [{'Amazon Video':...","{'stream_sd': [], 'rent_sd': [{'Amazon Video':...","{'stream_free': [], 'rent_free': [], 'buy_free...","{'stream_best': [], 'rent_best': [{'Amazon Vid...",None,https://www.justwatch.com/us/movie/laal-zameen
1,The Mannequin,(2022),"{'Rating': ' 4% ', 'Genres': 'Science-Fiction ...","{'stream_4k': [], 'rent_4k': [], 'buy_4k': []}","{'stream_hd': [], 'rent_hd': [{'Amazon Video':...","{'stream_sd': [{'Tubi TV': 'Ads '}], 'rent_sd'...","{'stream_free': [{'Tubi TV': 'Ads '}], 'rent_f...","{'stream_best': [{'Tubi TV': 'Ads '}], 'rent_b...",None,https://www.justwatch.com/us/movie/the-mannequin


In [ ]:
k_df = df['4k prices'].apply(pd.Series)
h_df = df['hd prices'].apply(pd.Series)
s_df = df['sd prices'].apply(pd.Series)
f_df = df['free'].apply(pd.Series)
b_df = df['best prices'].apply(pd.Series)
d_df = df['detail'].apply(pd.Series)
d_df

,Rating,Genres,Runtime,Director
0,4%,"Action & Adventure , Drama",2h 32min,Vivek Shrivastava
1,4%,"Science-Fiction , Animation , Action & Adventure",1h 4min,Jourin Hannah


In [ ]:
df = pd.concat([df, k_df.reindex(df.index)], axis=1)
df = pd.concat([df, h_df.reindex(df.index)], axis=1)
df = pd.concat([df, s_df.reindex(df.index)], axis=1)
df = pd.concat([df, f_df.reindex(df.index)], axis=1)
df = pd.concat([df, b_df.reindex(df.index)], axis=1)
df = pd.concat([df, d_df.reindex(df.index)], axis=1)

df_c = df.drop(['4k prices', 'hd prices', 'sd prices', 'free', 'best prices', 'detail'], axis=1)

In [ ]:
df_c

,title,year,imdb_link,tite url,stream_4k,rent_4k,buy_4k,stream_hd,rent_hd,buy_hd,...,stream_free,rent_free,buy_free,stream_best,rent_best,buy_best,Rating,Genres,Runtime,Director
0,Laal Zameen,(2022),None,https://www.justwatch.com/us/movie/laal-zameen,[],[],[],[],[{'Amazon Video': '$2.99 HD '}],[{'Amazon Video': '$9.99 HD '}],...,[],[],[],[],[{'Amazon Video': '$1.99 '}],[{'Amazon Video': '$7.99 '}],4%,"Action & Adventure , Drama",2h 32min,Vivek Shrivastava
1,The Mannequin,(2022),None,https://www.justwatch.com/us/movie/the-mannequin,[],[],[],[],[{'Amazon Video': '$2.99 HD '}],[{'Amazon Video': '$9.99 HD '}],...,[{'Tubi TV': 'Ads '}],[],[],[{'Tubi TV': 'Ads '}],[{'Amazon Video': '$1.99 '}],[{'Amazon Video': '$7.99 '}],4%,"Science-Fiction , Animation , Action & Adventure",1h 4min,Jourin Hannah


In [ ]:
# Convert dataframe to sql table                                   
df_c.to_sql('justwatch_data',  engine, if_exists='replace', index=False)

In [ ]:
df.to_csv("movies.csv")